In [44]:
from inspect import isclass
from typing import (
    Generic,
    List,
    Optional,
    Sequence,
    Tuple,
    TypeVar,
    Union,
    get_origin,
    get_args,
)
from types import UnionType
from pydantic.v1 import Field, BaseModel
from pydantic.v1.generics import GenericModel


class SubElement(BaseModel):
    x: int
    y: str


T = TypeVar("T")


class SubGenericElement(GenericModel, Generic[T]):
    value: T


# Original model with snake_case
class MyElement(BaseModel):
    # a: float
    # union1: int | str
    # union2: Union[int, str]
    opt1: int | None
    opt2: Optional[int]
    opt3: Union[int, None]
    # d: tuple[str]
    # e: Tuple[bool]
    # f: list[str]
    # g: List[int | str]
    # h: List[Union[int, str]]
    # dict1: dict[str, str]
    # sub: SubElement
    # sub2: SubGenericElement[SubElement]
    # sub3: SubGenericElement


# Create camelCase version
# MyElementNative = create_camel_model(MyElement)
# MyElementNative.__fields__


for field_name, field in MyElement.__fields__.items():
    print(f"Field {field_name} has type {field.type_}, outer type {field.outer_type_}")
    print(f"Origin: {get_origin(field.outer_type_)}")
    print(f"Args: {get_args(field.outer_type_)}")
    origin = get_origin(field.outer_type_)
    args = get_args(field.outer_type_)
    is_optional = not field.required
    is_list = origin is list
    is_tuple = origin is tuple
    is_object = origin is dict
    is_union = origin is Union or origin is UnionType
    # This check fails if we use the origin
    is_model = isclass(field.outer_type_) and issubclass(field.outer_type_, BaseModel)
    is_generic_model = (
        isclass(field.outer_type_)
        and issubclass(field.outer_type_, GenericModel)
        and not field.outer_type_.__concrete__
    )
    inner_types = [t for t in args if t is not type(None)]
    # print(f"Is optional = {is_optional}, is union = {is_union}, is list = {is_list}, is tuple = {is_tuple}, is_model = {is_model}, is_generic-model = {is_generic_model}")
    # print("Inner types:", inner_types)


# Now has fields: clientHeight, scrollWidth, contentEditable

Field opt1 has type <class 'int'>, outer type <class 'int'>
Origin: None
Args: ()
Field opt2 has type <class 'int'>, outer type <class 'int'>
Origin: None
Args: ()
Field opt3 has type <class 'int'>, outer type <class 'int'>
Origin: None
Args: ()


In [36]:
opt3 = MyElement.__fields__["opt3"]
opt3.outer_type_ == opt3.type_

True

In [53]:
from typing import Literal


t = int | str | None
get_origin(t), get_args(t)

(types.UnionType, (int, str, NoneType))

In [ ]:
from reflex_experiment.events import DOMEvents, ChangeEvent, FocusEvent
from reflex_experiment.elements import HTMLInputElement

input_change_event = FocusEvent[HTMLInputElement]
input_change_event.__fields__["related_target"].required


def test(evt: HTMLInputElement):
    evt.dict()

In [ ]:
import reflex as rx

rx.Base

In [2]:
from typing import Generic, TypeVar

from pydantic.v1 import BaseModel, validator, ValidationError
from pydantic.v1.generics import GenericModel

DataT = TypeVar("DataT")


class Error(BaseModel):
    code: int
    message: str


class DataModel(BaseModel):
    numbers: list[int]
    people: list[str]


class Response(GenericModel, Generic[DataT]):
    data: DataT | None = None
    error: Error | None = None

    @validator("error", always=True)
    def check_consistency(cls, v, values):
        if v is not None and values["data"] is not None:
            raise ValueError("must not provide both data and error")
        if v is None and values.get("data") is None:
            raise ValueError("must provide data or error")
        return v


data = DataModel(numbers=[1, 2, 3], people=[])
error = Error(code=404, message="Not found")

print(Response[int](data=1))
# > data=1 error=None
print(Response[str](data="value"))
# > data='value' error=None
print(Response[str](data="value").dict())
# > {'data': 'value', 'error': None}
print(Response[DataModel](data=data).dict())
"""
{
    'data': {'numbers': [1, 2, 3], 'people': []},
    'error': None,
}
"""
print(Response[DataModel](error=error).dict())
"""
{
    'data': None,
    'error': {'code': 404, 'message': 'Not found'},
}
"""
try:
    Response[int](data="value")
except ValidationError as e:
    print(e)
    """
    2 validation errors for Response[int]
    data
      value is not a valid integer (type=type_error.integer)
    error
      must provide data or error (type=value_error)
    """

data=1 error=None
data='value' error=None
{'data': 'value', 'error': None}
{'data': {'numbers': [1, 2, 3], 'people': []}, 'error': None}
{'data': None, 'error': {'code': 404, 'message': 'Not found'}}
2 validation errors for Response[int]
data
  value is not a valid integer (type=type_error.integer)
error
  must provide data or error (type=value_error)


In [5]:
from reflex_experiment.helpers import create_camel_model
from reflex_experiment.events import PointerEvent, TouchEvent
from reflex_experiment.elements import HTMLButtonElement

m = create_camel_model(TouchEvent[HTMLButtonElement], suffix="Native")
m.__fields__

ConfigError: Model 'HTMLAnchorElementNative' needs a discriminator field for key 'tag_name'